In [2]:
!pip install --upgrade pip

    100% |████████████████████████████████| 1.4MB 56.8MB/s ta 0:00:01
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1


In [11]:
!pip install --upgrade Pillow

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 2.1MB 30.4MB/s eta 0:00:01
  Found existing installation: Pillow 5.0.0
    Uninstalling Pillow-5.0.0:
      Successfully uninstalled Pillow-5.0.0


In [19]:
!pip install --upgrade scikit-image==0.15

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 26.3MB 65.2MB/s eta 0:00:01 1.9MB 65.2MB/s eta 0:00:01MB/s eta 0:00:01        | 10.4MB 65.2MB/s eta 0:00:01�████████████▊              | 14.6MB 65.2MB/s eta 0:00:01   |█████████████████████████████   | 23.9MB 65.2MB/s eta 0:00:01
ERROR: tensorlayer 2.1.0 has requirement wrapt==1.11.1, but you'll have wrapt 1.11.2 which is incompatible.
  Found existing installation: scikit-image 0.16.2
    Uninstalling scikit-image-0.16.2:
      Successfully uninstalled scikit-image-0.16.2


In [3]:
!pip install -r requirements.txt

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple


In [12]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [13]:
import os
import time
import random
import numpy as np
import scipy, multiprocessing
import tensorflow as tf
import tensorlayer as tl
from model import get_G, get_D
from config import config


In [14]:
###====================== HYPER-PARAMETERS ===========================###

batch_size = 8  # use 8 if your GPU memory is small, and change [4, 4] in tl.vis.save_images to [2, 4]
lr_init = config.TRAIN.lr_init
beta1 = config.TRAIN.beta1
## initialize G
n_epoch_init = config.TRAIN.n_epoch_init

n_epoch = config.TRAIN.n_epoch
lr_decay = config.TRAIN.lr_decay
decay_every = config.TRAIN.decay_every
shuffle_buffer_size = 128


In [15]:
# create folders to save result images and trained models
save_dir = "samples"
tl.files.exists_or_mkdir(save_dir)
checkpoint_dir = "checkpoint"
tl.files.exists_or_mkdir(checkpoint_dir)

[TL] [!] samples exists ...
[TL] [!] checkpoint exists ...


True

In [16]:
def get_train_data():
    # load dataset
    train_hr_img_list = sorted(tl.files.load_file_list(path=config.TRAIN.hr_img_path, regx='.*.png', printable=False))[0:200]

    ## If your machine have enough memory, please pre-load the entire train set.
    train_hr_imgs = tl.vis.read_images(train_hr_img_list, path=config.TRAIN.hr_img_path, n_threads=32)
        
    # dataset API and augmentation
    def generator_train():
        for img in train_hr_imgs:
            yield img
    def _map_fn_train(img):
        hr_patch = tf.image.random_crop(img, [384, 384, 3])
        hr_patch = hr_patch / (255. / 2.)
        hr_patch = hr_patch - 1.
        hr_patch = tf.image.random_flip_left_right(hr_patch)
        lr_patch = downscale_hr_patches(hr_patch)
        return lr_patch, hr_patch
    train_ds = tf.data.Dataset.from_generator(generator_train, output_types=(tf.float32))
    train_ds = train_ds.map(_map_fn_train, num_parallel_calls=multiprocessing.cpu_count())
        # train_ds = train_ds.repeat(n_epoch_init + n_epoch)
    train_ds = train_ds.shuffle(shuffle_buffer_size)
    train_ds = train_ds.prefetch(buffer_size=2)
    train_ds = train_ds.batch(batch_size)
        # value = train_ds.make_one_shot_iterator().get_next()
    return train_ds

def downscale_hr_patches(hr_patch):
    return tf.image.resize(hr_patch, size=[96, 96])

In [20]:
VGG = tl.models.vgg19(pretrained=True, end_with='pool4', mode='static')
train_ds = get_train_data()

[TL] Input  _inputlayer_3: [None, 224, 224, 3]
[TL] Lambda  scale: func: <function VGG_static.<locals>.<lambda> at 0x7f883f432c80>, len_weights: 0
[TL] Conv2d conv1_1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d conv1_2: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] MaxPool2d pool1: filter_size: (2, 2) strides: (2, 2) padding: SAME
[TL] Conv2d conv2_1: n_filter: 128 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d conv2_2: n_filter: 128 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] MaxPool2d pool2: filter_size: (2, 2) strides: (2, 2) padding: SAME
[TL] Conv2d conv3_1: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d conv3_2: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d conv3_3: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d conv3_4: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad

In [21]:
G = get_G((batch_size, 96, 96, 3))
D = get_D((batch_size, 384, 384, 3))


lr_v = tf.Variable(lr_init)
g_optimizer_init = tf.optimizers.Adam(lr_v, beta_1=beta1)
g_optimizer = tf.optimizers.Adam(lr_v, beta_1=beta1)
d_optimizer = tf.optimizers.Adam(lr_v, beta_1=beta1)

G.train()
D.train()
VGG.train()

[TL] Input  _inputlayer_4: (8, 96, 96, 3)
[TL] Conv2d conv2d_1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d conv2d_2: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_1: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d conv2d_3: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_2: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] Elementwise elementwise_1: fn: add act: No Activation
[TL] Conv2d conv2d_4: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_3: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d conv2d_5: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_4: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] Elementwise elementwise_2

[TL] BatchNorm batchnorm2d_34: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: False
[TL] Conv2d conv2d_40: n_filter: 256 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_35: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: False
[TL] Conv2d conv2d_41: n_filter: 512 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_36: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: False
[TL] Conv2d conv2d_42: n_filter: 1024 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_37: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: False
[TL] Conv2d conv2d_43: n_filter: 2048 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_38: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: False
[TL] Conv2d conv2d_44: n_filter: 1024 filter_size: (1, 1) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNo

In [22]:
from modelarts.session import Session
session = Session() 

In [ ]:
## initialize learning (G)
n_step_epoch = round(n_epoch_init // batch_size)
for epoch in range(n_epoch_init):
    for step, (lr_patchs, hr_patchs) in enumerate(train_ds):
        if lr_patchs.shape[0] != batch_size: # if the remaining data in this epoch < batch_size
            break
        step_time = time.time()
        with tf.GradientTape() as tape:
            fake_hr_patchs = G(lr_patchs)

            mse_loss = tl.cost.mean_squared_error(fake_hr_patchs, hr_patchs, is_mean=True)

        grad = tape.gradient(mse_loss, G.trainable_weights)
        g_optimizer_init.apply_gradients(zip(grad, G.trainable_weights))
        print("Epoch: [{}/{}] step: [{}/{}] time: {:.3f}s, mse: {:.3f} ".format(
            epoch, n_epoch_init, step, n_step_epoch, time.time() - step_time, mse_loss))
    if (epoch != 0) and (epoch % 10 == 0):
        tl.vis.save_images(fake_hr_patchs.numpy(), [2, 4], os.path.join(save_dir, 'train_g_init_{}.png'.format(epoch)))
        

Epoch: [0/100] step: [0/12] time: 1.163s, mse: 0.306 
Epoch: [0/100] step: [1/12] time: 0.460s, mse: 0.433 
Epoch: [0/100] step: [2/12] time: 0.457s, mse: 0.252 
Epoch: [0/100] step: [3/12] time: 0.470s, mse: 0.367 
Epoch: [0/100] step: [4/12] time: 0.447s, mse: 0.264 
Epoch: [0/100] step: [5/12] time: 0.466s, mse: 0.257 
Epoch: [0/100] step: [6/12] time: 0.479s, mse: 0.313 
Epoch: [0/100] step: [7/12] time: 0.453s, mse: 0.329 
Epoch: [0/100] step: [8/12] time: 0.366s, mse: 0.197 
Epoch: [0/100] step: [9/12] time: 0.357s, mse: 0.310 
Epoch: [0/100] step: [10/12] time: 0.532s, mse: 0.395 
Epoch: [0/100] step: [11/12] time: 0.372s, mse: 0.299 
Epoch: [0/100] step: [12/12] time: 0.360s, mse: 0.312 
Epoch: [0/100] step: [13/12] time: 0.358s, mse: 0.307 
Epoch: [0/100] step: [14/12] time: 0.358s, mse: 0.270 
Epoch: [0/100] step: [15/12] time: 0.368s, mse: 0.331 
Epoch: [0/100] step: [16/12] time: 0.372s, mse: 0.295 
Epoch: [0/100] step: [17/12] time: 0.359s, mse: 0.231 
Epoch: [0/100] step:

Epoch: [6/100] step: [1/12] time: 0.465s, mse: 0.058 
Epoch: [6/100] step: [2/12] time: 0.445s, mse: 0.028 
Epoch: [6/100] step: [3/12] time: 0.449s, mse: 0.042 
Epoch: [6/100] step: [4/12] time: 0.447s, mse: 0.057 
Epoch: [6/100] step: [5/12] time: 0.467s, mse: 0.058 
Epoch: [6/100] step: [6/12] time: 0.463s, mse: 0.043 
Epoch: [6/100] step: [7/12] time: 0.638s, mse: 0.040 
Epoch: [6/100] step: [8/12] time: 0.362s, mse: 0.038 
Epoch: [6/100] step: [9/12] time: 0.356s, mse: 0.065 
Epoch: [6/100] step: [10/12] time: 0.368s, mse: 0.043 
Epoch: [6/100] step: [11/12] time: 0.356s, mse: 0.042 
Epoch: [6/100] step: [12/12] time: 0.358s, mse: 0.035 
Epoch: [6/100] step: [13/12] time: 0.358s, mse: 0.026 
Epoch: [6/100] step: [14/12] time: 0.356s, mse: 0.057 
Epoch: [6/100] step: [15/12] time: 0.365s, mse: 0.067 
Epoch: [6/100] step: [16/12] time: 0.358s, mse: 0.058 
Epoch: [6/100] step: [17/12] time: 0.356s, mse: 0.054 
Epoch: [6/100] step: [18/12] time: 0.494s, mse: 0.065 
Epoch: [6/100] step

Epoch: [12/100] step: [1/12] time: 0.467s, mse: 0.042 
Epoch: [12/100] step: [2/12] time: 0.417s, mse: 0.017 
Epoch: [12/100] step: [3/12] time: 0.442s, mse: 0.030 
Epoch: [12/100] step: [4/12] time: 0.462s, mse: 0.037 
Epoch: [12/100] step: [5/12] time: 0.571s, mse: 0.036 
Epoch: [12/100] step: [6/12] time: 0.446s, mse: 0.029 
Epoch: [12/100] step: [7/12] time: 0.392s, mse: 0.038 
Epoch: [12/100] step: [8/12] time: 0.355s, mse: 0.031 
Epoch: [12/100] step: [9/12] time: 0.359s, mse: 0.051 
Epoch: [12/100] step: [10/12] time: 0.366s, mse: 0.041 
Epoch: [12/100] step: [11/12] time: 0.352s, mse: 0.034 
Epoch: [12/100] step: [12/12] time: 0.354s, mse: 0.044 
Epoch: [12/100] step: [13/12] time: 0.352s, mse: 0.023 
Epoch: [12/100] step: [14/12] time: 0.355s, mse: 0.045 
Epoch: [12/100] step: [15/12] time: 0.468s, mse: 0.039 
Epoch: [12/100] step: [16/12] time: 0.357s, mse: 0.019 
Epoch: [12/100] step: [17/12] time: 0.357s, mse: 0.039 
Epoch: [12/100] step: [18/12] time: 0.364s, mse: 0.042 
E

Epoch: [17/100] step: [24/12] time: 0.372s, mse: 0.034 
Epoch: [18/100] step: [0/12] time: 0.448s, mse: 0.043 
Epoch: [18/100] step: [1/12] time: 0.466s, mse: 0.039 
Epoch: [18/100] step: [2/12] time: 0.468s, mse: 0.021 
Epoch: [18/100] step: [3/12] time: 0.412s, mse: 0.030 
Epoch: [18/100] step: [4/12] time: 0.579s, mse: 0.039 
Epoch: [18/100] step: [5/12] time: 0.406s, mse: 0.034 
Epoch: [18/100] step: [6/12] time: 0.463s, mse: 0.032 
Epoch: [18/100] step: [7/12] time: 0.420s, mse: 0.029 
Epoch: [18/100] step: [8/12] time: 0.353s, mse: 0.025 
Epoch: [18/100] step: [9/12] time: 0.350s, mse: 0.043 
Epoch: [18/100] step: [10/12] time: 0.354s, mse: 0.039 
Epoch: [18/100] step: [11/12] time: 0.360s, mse: 0.040 
Epoch: [18/100] step: [12/12] time: 0.350s, mse: 0.057 
Epoch: [18/100] step: [13/12] time: 0.506s, mse: 0.031 
Epoch: [18/100] step: [14/12] time: 0.354s, mse: 0.042 
Epoch: [18/100] step: [15/12] time: 0.359s, mse: 0.051 
Epoch: [18/100] step: [16/12] time: 0.352s, mse: 0.023 
Ep

Epoch: [23/100] step: [22/12] time: 0.366s, mse: 0.055 
Epoch: [23/100] step: [23/12] time: 0.364s, mse: 0.024 
Epoch: [23/100] step: [24/12] time: 0.359s, mse: 0.018 
Epoch: [24/100] step: [0/12] time: 0.455s, mse: 0.033 
Epoch: [24/100] step: [1/12] time: 0.469s, mse: 0.040 
Epoch: [24/100] step: [2/12] time: 0.629s, mse: 0.022 
Epoch: [24/100] step: [3/12] time: 0.416s, mse: 0.014 
Epoch: [24/100] step: [4/12] time: 0.468s, mse: 0.034 
Epoch: [24/100] step: [5/12] time: 0.446s, mse: 0.025 
Epoch: [24/100] step: [6/12] time: 0.460s, mse: 0.028 
Epoch: [24/100] step: [7/12] time: 0.421s, mse: 0.029 
Epoch: [24/100] step: [8/12] time: 0.363s, mse: 0.030 
Epoch: [24/100] step: [9/12] time: 0.381s, mse: 0.040 
Epoch: [24/100] step: [10/12] time: 0.356s, mse: 0.036 
Epoch: [24/100] step: [11/12] time: 0.357s, mse: 0.028 
Epoch: [24/100] step: [12/12] time: 0.527s, mse: 0.023 
Epoch: [24/100] step: [13/12] time: 0.359s, mse: 0.019 
Epoch: [24/100] step: [14/12] time: 0.365s, mse: 0.053 
Ep

...

Epoch: [53/100] step: [12/12] time: 0.360s, mse: 0.027 
Epoch: [53/100] step: [13/12] time: 0.354s, mse: 0.024 
Epoch: [53/100] step: [14/12] time: 0.351s, mse: 0.028 
Epoch: [53/100] step: [15/12] time: 0.352s, mse: 0.020 
Epoch: [53/100] step: [16/12] time: 0.354s, mse: 0.019 
Epoch: [53/100] step: [17/12] time: 0.472s, mse: 0.023 
Epoch: [53/100] step: [18/12] time: 0.356s, mse: 0.026 
Epoch: [53/100] step: [19/12] time: 0.358s, mse: 0.012 
Epoch: [53/100] step: [20/12] time: 0.352s, mse: 0.027 
Epoch: [53/100] step: [21/12] time: 0.352s, mse: 0.016 
Epoch: [53/100] step: [22/12] time: 0.348s, mse: 0.052 
Epoch: [53/100] step: [23/12] time: 0.348s, mse: 0.017 
Epoch: [53/100] step: [24/12] time: 0.353s, mse: 0.023 
Epoch: [54/100] step: [0/12] time: 0.421s, mse: 0.024 
Epoch: [54/100] step: [1/12] time: 0.466s, mse: 0.032 
Epoch: [54/100] step: [2/12] time: 0.455s, mse: 0.019 
Epoch: [54/100] step: [3/12] time: 0.519s, mse: 0.025 
Epoch: [54/100] step: [4/12] time: 0.428s, mse: 0.04

Epoch: [59/100] step: [10/12] time: 0.359s, mse: 0.021 
Epoch: [59/100] step: [11/12] time: 0.354s, mse: 0.013 
Epoch: [59/100] step: [12/12] time: 0.366s, mse: 0.020 
Epoch: [59/100] step: [13/12] time: 0.360s, mse: 0.014 
Epoch: [59/100] step: [14/12] time: 0.360s, mse: 0.024 
Epoch: [59/100] step: [15/12] time: 0.581s, mse: 0.036 
Epoch: [59/100] step: [16/12] time: 0.355s, mse: 0.017 
Epoch: [59/100] step: [17/12] time: 0.363s, mse: 0.019 
Epoch: [59/100] step: [18/12] time: 0.354s, mse: 0.023 
Epoch: [59/100] step: [19/12] time: 0.356s, mse: 0.013 
Epoch: [59/100] step: [20/12] time: 0.358s, mse: 0.023 
Epoch: [59/100] step: [21/12] time: 0.354s, mse: 0.014 
Epoch: [59/100] step: [22/12] time: 0.357s, mse: 0.037 
Epoch: [59/100] step: [23/12] time: 0.362s, mse: 0.016 
Epoch: [59/100] step: [24/12] time: 0.352s, mse: 0.022 
Epoch: [60/100] step: [0/12] time: 0.608s, mse: 0.026 
Epoch: [60/100] step: [1/12] time: 0.435s, mse: 0.027 
Epoch: [60/100] step: [2/12] time: 0.450s, mse: 0.

Epoch: [65/100] step: [8/12] time: 0.363s, mse: 0.022 
Epoch: [65/100] step: [9/12] time: 0.353s, mse: 0.025 
Epoch: [65/100] step: [10/12] time: 0.350s, mse: 0.024 
Epoch: [65/100] step: [11/12] time: 0.362s, mse: 0.016 
Epoch: [65/100] step: [12/12] time: 0.388s, mse: 0.025 
Epoch: [65/100] step: [13/12] time: 0.376s, mse: 0.012 
Epoch: [65/100] step: [14/12] time: 0.521s, mse: 0.032 
Epoch: [65/100] step: [15/12] time: 0.363s, mse: 0.030 
Epoch: [65/100] step: [16/12] time: 0.355s, mse: 0.017 
Epoch: [65/100] step: [17/12] time: 0.355s, mse: 0.033 
Epoch: [65/100] step: [18/12] time: 0.351s, mse: 0.021 
Epoch: [65/100] step: [19/12] time: 0.356s, mse: 0.014 
Epoch: [65/100] step: [20/12] time: 0.348s, mse: 0.018 
Epoch: [65/100] step: [21/12] time: 0.355s, mse: 0.015 
Epoch: [65/100] step: [22/12] time: 0.364s, mse: 0.043 
Epoch: [65/100] step: [23/12] time: 0.495s, mse: 0.013 
Epoch: [65/100] step: [24/12] time: 0.355s, mse: 0.011 
Epoch: [66/100] step: [0/12] time: 0.479s, mse: 0.

Epoch: [71/100] step: [6/12] time: 0.428s, mse: 0.018 
Epoch: [71/100] step: [7/12] time: 0.422s, mse: 0.020 
Epoch: [71/100] step: [8/12] time: 0.358s, mse: 0.021 
Epoch: [71/100] step: [9/12] time: 0.353s, mse: 0.031 
Epoch: [71/100] step: [10/12] time: 0.351s, mse: 0.027 
Epoch: [71/100] step: [11/12] time: 0.351s, mse: 0.016 
Epoch: [71/100] step: [12/12] time: 0.509s, mse: 0.022 
Epoch: [71/100] step: [13/12] time: 0.355s, mse: 0.013 
Epoch: [71/100] step: [14/12] time: 0.358s, mse: 0.028 
Epoch: [71/100] step: [15/12] time: 0.352s, mse: 0.028 
Epoch: [71/100] step: [16/12] time: 0.349s, mse: 0.022 
Epoch: [71/100] step: [17/12] time: 0.353s, mse: 0.030 
Epoch: [71/100] step: [18/12] time: 0.361s, mse: 0.026 
Epoch: [71/100] step: [19/12] time: 0.355s, mse: 0.011 
Epoch: [71/100] step: [20/12] time: 0.357s, mse: 0.015 
Epoch: [71/100] step: [21/12] time: 0.356s, mse: 0.018 
Epoch: [71/100] step: [22/12] time: 0.538s, mse: 0.050 
Epoch: [71/100] step: [23/12] time: 0.354s, mse: 0.0

Epoch: [77/100] step: [4/12] time: 0.455s, mse: 0.028 
Epoch: [77/100] step: [5/12] time: 0.433s, mse: 0.019 
Epoch: [77/100] step: [6/12] time: 0.438s, mse: 0.020 
Epoch: [77/100] step: [7/12] time: 0.434s, mse: 0.021 
Epoch: [77/100] step: [8/12] time: 0.360s, mse: 0.015 
Epoch: [77/100] step: [9/12] time: 0.363s, mse: 0.020 
Epoch: [77/100] step: [10/12] time: 0.489s, mse: 0.020 
Epoch: [77/100] step: [11/12] time: 0.353s, mse: 0.015 
Epoch: [77/100] step: [12/12] time: 0.356s, mse: 0.029 
Epoch: [77/100] step: [13/12] time: 0.354s, mse: 0.009 
Epoch: [77/100] step: [14/12] time: 0.352s, mse: 0.023 
Epoch: [77/100] step: [15/12] time: 0.357s, mse: 0.018 
Epoch: [77/100] step: [16/12] time: 0.363s, mse: 0.016 
Epoch: [77/100] step: [17/12] time: 0.362s, mse: 0.024 
Epoch: [77/100] step: [18/12] time: 0.363s, mse: 0.026 
Epoch: [77/100] step: [19/12] time: 0.351s, mse: 0.014 
Epoch: [77/100] step: [20/12] time: 0.520s, mse: 0.014 
Epoch: [77/100] step: [21/12] time: 0.353s, mse: 0.015

In [ ]:
G_base.save_weights(os.path.join(checkpoint_dir, 'g-initial.h5'))
session.upload_data(bucket_path="/mipt-course/tendai/srgan-checkpoints-base", path=checkpoint_dir)

In [ ]:
## adversarial learning (G, D)
n_step_epoch = round(n_epoch // batch_size)
for epoch in range(n_epoch):
    for step, (lr_patchs, hr_patchs) in enumerate(train_ds):
        if lr_patchs.shape[0] != batch_size: # if the remaining data in this epoch < batch_size
            break
        step_time = time.time()
        with tf.GradientTape(persistent=True) as tape:
            fake_patchs = G(lr_patchs)
            logits_fake = D(fake_patchs)
            logits_real = D(hr_patchs)
            feature_fake = VGG((fake_patchs+1)/2.) # the pre-trained VGG uses the input range of [0, 1]
            feature_real = VGG((hr_patchs+1)/2.)
            d_loss1 = tl.cost.sigmoid_cross_entropy(logits_real, tf.ones_like(logits_real))
            d_loss2 = tl.cost.sigmoid_cross_entropy(logits_fake, tf.zeros_like(logits_fake))
            d_loss = d_loss1 + d_loss2
            g_gan_loss = 1e-3 * tl.cost.sigmoid_cross_entropy(logits_fake, tf.ones_like(logits_fake))

            mse_loss = tl.cost.mean_squared_error(fake_patchs, hr_patchs, is_mean=True)
            
            vgg_loss = 2e-6 * tl.cost.mean_squared_error(feature_fake, feature_real, is_mean=True)
            g_loss = mse_loss + vgg_loss + g_gan_loss
        grad = tape.gradient(g_loss, G.trainable_weights)
        g_optimizer.apply_gradients(zip(grad, G.trainable_weights))
        grad = tape.gradient(d_loss, D.trainable_weights)
        d_optimizer.apply_gradients(zip(grad, D.trainable_weights))
        
        print("Epoch: [{}/{}] step: [{}/{}] time: {:.3f}s, g_loss(mse:{:.3f}, vgg:{:.3f}, adv:{:.3f}) d_loss: {:.3f}".format(
            epoch, n_epoch_init, step, n_step_epoch, time.time() - step_time, mse_loss, vgg_loss, g_gan_loss, d_loss))

    # update the learning rate
    if epoch != 0 and (epoch % decay_every == 0):
        new_lr_decay = lr_decay**(epoch // decay_every)
        lr_v.assign(lr_init * new_lr_decay)
        log = " ** new learning rate: %f (for GAN)" % (lr_init * new_lr_decay)
        print(log)

    if (epoch != 0) and (epoch % 10 == 0):
        tl.vis.save_images(fake_patchs.numpy(), [2, 4], os.path.join(save_dir, 'train_g_{}.png'.format(epoch)))
        G.save_weights(os.path.join(checkpoint_dir, 'g-{epoch}.h5'.format(epoch=epoch)))
        D.save_weights(os.path.join(checkpoint_dir, 'd-{epoch}.h5'.format(epoch=epoch)))
        session.upload_data(bucket_path="/mipt-course/tendai/srgan-samples-base/samples", path=os.path.join(save_dir, 'train_g_base_{}.png'.format(epoch)))
        session.upload_data(bucket_path="/mipt-course/tendai/srgan-checkpoints-base", path=checkpoint_dir)
        